## Group Project LLM

- r=2,4,8,16, epoch=30
- seed=42
- evaluation:
    - accuracy, f1, precision, recall
    - efficiency (time, trainable parameters, trainable paramters ratio, early stopping)

In [1]:
import warnings
warnings.filterwarnings("default", module="__main__")
warnings.filterwarnings("ignore", module=".*")

### Base Model: DistilBERT

In [2]:
# ================== BASELINE DISTILBERT ================
# Commented out - not needed for Sparse LoRA experiments

import os, time, random
import numpy as np
import torch

from datasets import load_dataset
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    set_seed
)
import evaluate

# Force to use only GPU 1
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Print GPU info
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"[GPU Info] {num_gpus} GPU(s) available")
    for i in range(num_gpus):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("[GPU Info] No GPU available, using CPU")

SEED = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATASET = "stanfordnlp/sst2"
TEXT_COL = "sentence"
LABEL_COL = "label"
NUM_EPOCHS = 30
BATCH_SIZE = 16
LR = 5e-4

'''
for IMDB dataset:
DATASET = "imdb"
TEXT_COL = "text"
LABEL_COL = "label"
'''

def set_all_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    set_seed(seed)

set_all_seeds(SEED)

# -------- Load dataset and split (8:1:1) --------
raw = load_dataset(DATASET)
train_full = raw["train"]

train_temp = train_full.train_test_split(test_size=0.2, seed=SEED)
train_ds = train_temp["train"]
temp = train_temp["test"]

val_test = temp.train_test_split(test_size=0.5, seed=SEED)
val_ds = val_test["train"]
test_ds = val_test["test"]


# -------- Tokenization --------
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
def preprocess(x):
    return tokenizer(x[TEXT_COL], truncation=True, max_length=128)

train_ds = train_ds.map(preprocess, batched=True)
val_ds   = val_ds.map(preprocess, batched=True)
test_ds  = test_ds.map(preprocess, batched=True)

train_ds = train_ds.rename_column(LABEL_COL, "labels")
val_ds   = val_ds.rename_column(LABEL_COL, "labels")
test_ds  = test_ds.rename_column(LABEL_COL, "labels")

cols = ["input_ids","attention_mask","labels"]
train_ds.set_format(type="torch", columns=cols)
val_ds.set_format(type="torch", columns=cols)
test_ds.set_format(type="torch", columns=cols)

collator = DataCollatorWithPadding(tokenizer)

# -------- Metrics --------
acc = evaluate.load("accuracy")
f1 = evaluate.load("f1")
prec = evaluate.load("precision")
rec = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels, average="binary")["f1"],
        "precision": prec.compute(predictions=preds, references=labels, average="binary")["precision"],
        "recall": rec.compute(predictions=preds, references=labels, average="binary")["recall"],
    }

# -------- Model --------
# COMMENTED OUT: Baseline full fine-tuning (not needed for Sparse LoRA)
'''
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
).to(DEVICE)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
ratio = trainable_params / total_params

print(f"Baseline: total={total_params}, trainable={trainable_params}, ratio={ratio:.4%}")

# -------- Train --------
args = TrainingArguments(
    output_dir="./baseline_distilbert",
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    seed=SEED,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

start = time.time()
trainer.train()
end = time.time()

print(f"Baseline training time: {end-start:.2f}s")
print("Eval:", trainer.evaluate(test_ds))
print("Training history:", trainer.state.log_history)
'''

[GPU Info] 1 GPU(s) available
  GPU 0: NVIDIA RTX 5000 Ada Generation


'\nmodel = DistilBertForSequenceClassification.from_pretrained(\n    "distilbert-base-uncased", num_labels=2\n).to(DEVICE)\n\ntotal_params = sum(p.numel() for p in model.parameters())\ntrainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)\nratio = trainable_params / total_params\n\nprint(f"Baseline: total={total_params}, trainable={trainable_params}, ratio={ratio:.4%}")\n\n# -------- Train --------\nargs = TrainingArguments(\n    output_dir="./baseline_distilbert",\n    num_train_epochs=NUM_EPOCHS,\n    per_device_train_batch_size=BATCH_SIZE,\n    per_device_eval_batch_size=BATCH_SIZE,\n    learning_rate=LR,\n    eval_strategy="epoch",\n    save_strategy="epoch",\n    logging_strategy="epoch",\n    load_best_model_at_end=True,\n    metric_for_best_model="f1",\n    greater_is_better=True,\n    seed=SEED,\n    report_to="none",\n)\n\ntrainer = Trainer(\n    model=model,\n    args=args,\n    train_dataset=train_ds,\n    eval_dataset=val_ds,\n    data_collator=co

In [3]:
# COMMENTED OUT: Baseline model saving and evaluation
'''
import pandas as pd
# -------- Final Evaluation --------
final_metrics = trainer.evaluate(test_ds)

# -------- Save metrics --------
os.makedirs("./baseline_distilbert", exist_ok=True)
with open("./baseline_distilbert/final_metrics.json", "w") as f:
    json.dump(final_metrics, f, indent=4)

print("Saved final metrics to baseline_distilbert/final_metrics.json")

# -------- Save model --------
trainer.save_model("./baseline_distilbert/final_model")
print("Saved model to baseline_distilbert/final_model")

# -------- Training history --------
log_history = trainer.state.log_history
df_logs = pd.DataFrame(trainer.state.log_history)
# Separate clean tables
df_train = df_logs[df_logs["loss"].notnull()].reset_index(drop=True)
df_eval  = df_logs[df_logs["eval_loss"].notnull()].reset_index(drop=True)

df_train.to_csv("./baseline_distilbert/train_log.csv", index=False)
df_eval.to_csv("./baseline_distilbert/eval_log.csv", index=False)
'''

'\nimport pandas as pd\n# -------- Final Evaluation --------\nfinal_metrics = trainer.evaluate(test_ds)\n\n# -------- Save metrics --------\nos.makedirs("./baseline_distilbert", exist_ok=True)\nwith open("./baseline_distilbert/final_metrics.json", "w") as f:\n    json.dump(final_metrics, f, indent=4)\n\nprint("Saved final metrics to baseline_distilbert/final_metrics.json")\n\n# -------- Save model --------\ntrainer.save_model("./baseline_distilbert/final_model")\nprint("Saved model to baseline_distilbert/final_model")\n\n# -------- Training history --------\nlog_history = trainer.state.log_history\ndf_logs = pd.DataFrame(trainer.state.log_history)\n# Separate clean tables\ndf_train = df_logs[df_logs["loss"].notnull()].reset_index(drop=True)\ndf_eval  = df_logs[df_logs["eval_loss"].notnull()].reset_index(drop=True)\n\ndf_train.to_csv("./baseline_distilbert/train_log.csv", index=False)\ndf_eval.to_csv("./baseline_distilbert/eval_log.csv", index=False)\n'

In [4]:
# COMMENTED OUT: Baseline visualization
'''
import matplotlib.pyplot as plt

df_eval_clean = df_eval.groupby("epoch").last().reset_index()

plt.figure(figsize=(8,5))
plt.plot(df_train["epoch"], df_train["loss"], label="Train Loss")
plt.plot(df_eval_clean["epoch"], df_eval_clean["eval_loss"], label="Eval Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training: Train vs Eval Loss")
plt.legend()
plt.grid()
plt.savefig("./baseline_distilbert/loss_curve.png", dpi=300)
plt.show()


plt.figure(figsize=(8,5))
plt.plot(df_eval_clean["epoch"], df_eval_clean["eval_accuracy"], label="Eval Accuracy")
plt.plot(df_eval_clean["epoch"], df_eval_clean["eval_f1"], label="Eval F1")
plt.xlabel("Epoch")
plt.ylabel("Metric")
plt.title("Evaluation: Accuracy & F1")
plt.legend()
plt.grid()
plt.savefig("./baseline_distilbert/metric_curve.png", dpi=300)
plt.show()
'''

'\nimport matplotlib.pyplot as plt\n\ndf_eval_clean = df_eval.groupby("epoch").last().reset_index()\n\nplt.figure(figsize=(8,5))\nplt.plot(df_train["epoch"], df_train["loss"], label="Train Loss")\nplt.plot(df_eval_clean["epoch"], df_eval_clean["eval_loss"], label="Eval Loss")\nplt.xlabel("Epoch")\nplt.ylabel("Loss")\nplt.title("Training: Train vs Eval Loss")\nplt.legend()\nplt.grid()\nplt.savefig("./baseline_distilbert/loss_curve.png", dpi=300)\nplt.show()\n\n\nplt.figure(figsize=(8,5))\nplt.plot(df_eval_clean["epoch"], df_eval_clean["eval_accuracy"], label="Eval Accuracy")\nplt.plot(df_eval_clean["epoch"], df_eval_clean["eval_f1"], label="Eval F1")\nplt.xlabel("Epoch")\nplt.ylabel("Metric")\nplt.title("Evaluation: Accuracy & F1")\nplt.legend()\nplt.grid()\nplt.savefig("./baseline_distilbert/metric_curve.png", dpi=300)\nplt.show()\n'

### Sparse LoRA

In [5]:
# ================== SPARSE LoRA MODEL =================

from typing import Dict, Any, List, Optional
import math
import json
import pandas as pd
from peft import LoraConfig, get_peft_model

# -------- Sparse LoRA config --------
RANKS: List[int] = [2, 4, 8, 16]
L1_LAMBDA = 1e-5   # sparsity strength for LoRA weights


def count_trainable_params(model: torch.nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def count_total_params(model: torch.nn.Module) -> int:
    return sum(p.numel() for p in model.parameters())


def compute_lora_sparsity(model: torch.nn.Module, threshold: float = 1e-3) -> float:
    """
    Approximate sparsity: fraction of LoRA parameters with |w| < threshold.
    """
    total = 0
    near_zero = 0
    for name, param in model.named_parameters():
        if "lora_" in name and param.requires_grad:
            data = param.detach().abs()
            total += data.numel()
            near_zero += (data < threshold).sum().item()
    return near_zero / total if total > 0 else math.nan


from transformers import TrainerCallback

class MetricsCallback(TrainerCallback):
    """
    Callback to track metrics and implement early stopping.
    """
    def __init__(self, early_stop_patience=3):
        self.epoch_times = []
        self.epoch_start_time = None
        self.epoch_f1s = []
        self.best_f1 = 0.0
        self.best_f1_epoch = None
        self.logged_epochs = set()
        # Early stopping: stop when validation F1 fails to improve over best_f1 for `early_stop_patience` consecutive epochs
        self.early_stop_patience = early_stop_patience
        self.epochs_without_improvement = 0
        self.early_stopped = False

    def on_epoch_begin(self, args, state, control, **kwargs):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, args, state, control, **kwargs):
        if self.epoch_start_time is not None:
            epoch_time = time.time() - self.epoch_start_time
            self.epoch_times.append(epoch_time)

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        if 'eval_f1' in metrics and state.epoch > 0:
            f1 = metrics['eval_f1']
            current_epoch = int(state.epoch)
            
            if current_epoch not in self.logged_epochs:
                self.logged_epochs.add(current_epoch)
                self.epoch_f1s.append(f1)

                # Update best F1 and reset counter if improved
                if f1 > self.best_f1:
                    self.best_f1 = f1
                    self.best_f1_epoch = current_epoch
                    self.epochs_without_improvement = 0
                else:
                    # F1 did not improve over best_f1
                    self.epochs_without_improvement += 1

                # Trigger early stopping if no improvement for patience epochs
                if self.epochs_without_improvement >= self.early_stop_patience and not self.early_stopped:
                    self.early_stopped = True
                    # Request Trainer to stop training after this evaluation
                    control.should_training_stop = True
                    print(f"\n[Early Stopping] No improvement over best F1 for {self.epochs_without_improvement} consecutive epochs. Stopping training at epoch {current_epoch}.")
                    print(f"[Early Stopping Info] Best F1: {self.best_f1:.4f} at epoch {self.best_f1_epoch}\n")


class SparseLoraTrainer(Trainer):
    """
    Trainer with L1 penalty only on LoRA parameters.
    """
    def __init__(self, l1_lambda: float = 0.0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.l1_lambda = l1_lambda

    def compute_loss(
        self,
        model,
        inputs,
        return_outputs: bool = False,
        num_items_in_batch: Optional[int] = None,
    ):
        outputs = model(**inputs)
        loss = outputs.loss

        if self.l1_lambda > 0:
            l1_reg = 0.0
            for name, param in model.named_parameters():
                if "lora_" in name and param.requires_grad:
                    l1_reg = l1_reg + param.abs().sum()
            loss = loss + self.l1_lambda * l1_reg

        return (loss, outputs) if return_outputs else loss


results_per_rank: List[Dict[str, Any]] = []

for r in RANKS:
    print("\n" + "=" * 80)
    print(f"Training Sparse LoRA DistilBERT with rank = {r}, epochs = {NUM_EPOCHS}")
    print("=" * 80)

    set_all_seeds(SEED)

    # Check for existing checkpoint
    output_dir = f"/hpc/group/xielab/hl385/LoRA/sparse_lora_rank{r}"
    checkpoint_dir = None
    resume_from_checkpoint = False  # Set to True to resume from checkpoint
    if resume_from_checkpoint and os.path.exists(output_dir):
        checkpoints = [d for d in os.listdir(output_dir) if d.startswith("checkpoint-")]
        if checkpoints:
            checkpoints.sort(key=lambda x: int(x.split("-")[1]))
            checkpoint_dir = os.path.join(output_dir, checkpoints[-1])
            print(f"Found existing checkpoint: {checkpoint_dir}")
            print(f"Resuming training from this checkpoint...\n")

    # Reset GPU memory stats
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()

    # Base DistilBERT for this rank
    base_model = DistilBertForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        num_labels=2,
    )

    # LoRA config: attention projections in DistilBERT
    lora_config = LoraConfig(
        r=r,
        lora_alpha=2 * r,
        lora_dropout=0.1,
        bias="none",
        task_type="SEQ_CLS",    # sequence classification
        target_modules=["q_lin", "k_lin", "v_lin", "out_lin"],
    )

    lora_model = get_peft_model(base_model, lora_config)
    lora_model.to(DEVICE)

    total_params = count_total_params(lora_model)
    trainable_params = count_trainable_params(lora_model)
    param_ratio = trainable_params / total_params

    print(f"[Rank {r}] total params: {total_params:,}")
    print(f"[Rank {r}] trainable params: {trainable_params:,}")
    print(f"[Rank {r}] trainable params ratio (trainable / total): {param_ratio:.4%}")

    os.makedirs(output_dir, exist_ok=True)

    training_args_lora = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        learning_rate=LR,
        weight_decay=0.01,
        logging_steps=100,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        save_total_limit=2,
        seed=SEED,
        report_to="none",
        disable_tqdm=False,
    )

    # Create metrics callback
    metrics_callback = MetricsCallback(early_stop_patience=3)

    trainer = SparseLoraTrainer(
        l1_lambda=L1_LAMBDA,
        model=lora_model,
        args=training_args_lora,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=collator,
        compute_metrics=compute_metrics,
        callbacks=[metrics_callback],
    )

    start_time = time.time()
    trainer.train(resume_from_checkpoint=checkpoint_dir if checkpoint_dir else None)
    end_time = time.time()
    total_train_time = end_time - start_time
    
    # Calculate metrics
    avg_epoch_time = sum(metrics_callback.epoch_times) / len(metrics_callback.epoch_times) if metrics_callback.epoch_times else 0.0
    
    # Get peak GPU memory
    peak_gpu_memory_mb = 0.0
    if torch.cuda.is_available():
        peak_gpu_memory_mb = torch.cuda.max_memory_allocated() / (1024 ** 2)
        num_gpus = torch.cuda.device_count()
        current_device = torch.cuda.current_device()
        device_name = torch.cuda.get_device_name(current_device)
        print(f"\n[GPU Info] Used {num_gpus} GPU(s), Current device: {current_device} ({device_name})")

    # --- final evals (using best model loaded by load_best_model_at_end=True) ---
    val_metrics = trainer.evaluate(eval_dataset=val_ds)
    test_metrics = trainer.evaluate(eval_dataset=test_ds)
    lora_sparsity = compute_lora_sparsity(lora_model, threshold=1e-3)
    
    # Print results in summary table format
    print(f"\n{'='*80}")
    print(f"Results for rank = {r}:")
    print(f"  Test Accuracy: {test_metrics.get('eval_accuracy', 0.0):.4f}")
    print(f"  Test F1: {test_metrics.get('eval_f1', 0.0):.4f}")
    print(f"  Validation Accuracy: {val_metrics.get('eval_accuracy', 0.0):.4f}")
    print(f"  Validation F1: {val_metrics.get('eval_f1', 0.0):.4f}")
    print(f"  Total Training Time: {total_train_time:.2f}s")
    print(f"  Average Epoch Time: {avg_epoch_time:.2f}s")
    print(f"  Best F1 Epoch: {metrics_callback.best_f1_epoch}")
    print(f"  Best Val F1: {metrics_callback.best_f1:.4f}")
    print(f"  Early Stopped: {metrics_callback.early_stopped}")
    if metrics_callback.early_stopped:
        print(f"  Stop Info: epochs_without_improvement={metrics_callback.epochs_without_improvement}")
    print(f"  Peak GPU Memory: {peak_gpu_memory_mb:.2f} MB")
    print(f"  LoRA Sparsity (<1e-3): {lora_sparsity:.2%}")
    print(f"{'='*80}\n")

    # ==========================
    # SAVE METRICS / MODEL / LOG
    # ==========================
    # 1) save metrics
    metrics_payload = {
        "rank": r,
        "total_train_time_sec": total_train_time,
        "avg_epoch_time_sec": avg_epoch_time,
        "best_f1_epoch": metrics_callback.best_f1_epoch,
        "best_val_f1": float(metrics_callback.best_f1),
        "early_stopped": metrics_callback.early_stopped,
        "peak_gpu_memory_mb": peak_gpu_memory_mb,
        "total_params": int(total_params),
        "trainable_params": int(trainable_params),
        "param_ratio": float(param_ratio),
        "lora_sparsity_<1e-3": float(lora_sparsity),
        "val_metrics": val_metrics,
        "test_metrics": test_metrics,
    }
    with open(os.path.join(output_dir, "final_metrics.json"), "w") as f:
        json.dump(metrics_payload, f, indent=4)

    # 2) save final model (best checkpoint)
    final_model_dir = os.path.join(output_dir, "final_model")
    trainer.save_model(final_model_dir)  # saves model + config
    tokenizer.save_pretrained(final_model_dir)  # save tokenizer too
    print(f"[Rank {r}] Saved model to {final_model_dir}")

    # 3) save training history
    log_history = trainer.state.log_history
    with open(os.path.join(output_dir, "log_history.json"), "w") as f:
        json.dump(log_history, f, indent=4)
    print(f"[Rank {r}] Saved log history to log_history.json")

    # --- store in-memory summary for printing ---
    results_per_rank.append(
        {
            "rank": r,
            "final_test_accuracy": test_metrics.get('eval_accuracy', 0.0),
            "final_test_f1": test_metrics.get('eval_f1', 0.0),
            "final_val_accuracy": val_metrics.get('eval_accuracy', 0.0),
            "final_val_f1": val_metrics.get('eval_f1', 0.0),
            "total_parameters": total_params,
            "trainable_parameters": trainable_params,
            "trainable_percentage": param_ratio * 100,
            "total_training_time": total_train_time,
            "average_epoch_time": avg_epoch_time,
            "peak_gpu_memory_mb": peak_gpu_memory_mb,
            "sparsity": lora_sparsity,
            "best_f1_epoch": metrics_callback.best_f1_epoch,
            "best_val_f1": metrics_callback.best_f1,
            "early_stopped": metrics_callback.early_stopped,
        }
    )

# Generate and save summary table
print("\n\n" + "=" * 80)
print("GENERATING SUMMARY TABLE")
print("=" * 80)

summary_data = []
for result in results_per_rank:
    summary_data.append({
        "Rank": result['rank'],
        "Test Acc": f"{result['final_test_accuracy']:.4f}",
        "Test F1": f"{result['final_test_f1']:.4f}",
        "Val Acc": f"{result['final_val_accuracy']:.4f}",
        "Val F1": f"{result['final_val_f1']:.4f}",
        "Best Val F1": f"{result['best_val_f1']:.4f}",
        "Best F1 Epoch": result['best_f1_epoch'],
        "Trainable Params": f"{result['trainable_parameters']:,}",
        "Trainable %": f"{result['trainable_percentage']:.2f}%",
        "Total Time (s)": f"{result['total_training_time']:.2f}",
        "Avg Epoch (s)": f"{result['average_epoch_time']:.2f}",
        "Early Stopped": "Yes" if result['early_stopped'] else "No",
        "Peak GPU (MB)": f"{result['peak_gpu_memory_mb']:.2f}",
        "Sparsity (<1e-3)": f"{result['sparsity']*100:.2f}%"
    })

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*80)
print("BENCHMARK SUMMARY - Sparse LoRA")
print("="*80)
print(summary_df.to_string(index=False))

# Save summary to CSV
csv_filename = "SparseLoRA_SST2_benchmark_summary.csv"
summary_df.to_csv(csv_filename, index=False)
print(f"\n✓ Summary saved to '{csv_filename}'")

# Save complete results
with open("sparse_lora_all_results.json", "w") as f:
    json.dump(results_per_rank, f, indent=2)
print("✓ Complete results saved to 'sparse_lora_all_results.json'")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Training Sparse LoRA DistilBERT with rank = 2, epochs = 30
[Rank 2] total params: 67,620,868
[Rank 2] trainable params: 665,858
[Rank 2] trainable params ratio (trainable / total): 0.9847%


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.278700,0.222086,0.922197,0.927584,0.928354,0.926816
2,0.232400,0.210075,0.927988,0.931815,0.949026,0.915217
3,0.241300,0.198828,0.936154,0.940410,0.943811,0.937034
4,0.180500,0.235969,0.936897,0.941565,0.937568,0.945595
5,0.184200,0.237557,0.938976,0.942348,0.957526,0.927644
6,0.206200,0.207979,0.939569,0.944070,0.939551,0.948633
7,0.166100,0.225069,0.940906,0.945134,0.943573,0.946700
8,0.169300,0.238490,0.942242,0.946085,0.949638,0.942557
9,0.156800,0.241769,0.939866,0.944161,0.942731,0.945595
10,0.154100,0.255681,0.942242,0.945511,0.959352,0.932063



[Early Stopping] No improvement over best F1 for 3 consecutive epochs. Stopping training at epoch 11.
[Early Stopping Info] Best F1: 0.9461 at epoch 8


[GPU Info] Used 1 GPU(s), Current device: 0 (NVIDIA RTX 5000 Ada Generation)



Results for rank = 2:
  Test Accuracy: 0.9424
  Test F1: 0.9487
  Validation Accuracy: 0.9422
  Validation F1: 0.9461
  Total Training Time: 1627.23s
  Average Epoch Time: 142.58s
  Best F1 Epoch: 8
  Best Val F1: 0.9461
  Early Stopped: True
  Stop Info: epochs_without_improvement=3
  Peak GPU Memory: 590.62 MB
  LoRA Sparsity (<1e-3): 2.86%



...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Rank 8] Saved model to /hpc/group/xielab/hl385/LoRA/sparse_lora_rank8/final_model
[Rank 8] Saved log history to log_history.json

Training Sparse LoRA DistilBERT with rank = 16, epochs = 30
[Rank 16] total params: 68,136,964
[Rank 16] trainable params: 1,181,954
[Rank 16] trainable params ratio (trainable / total): 1.7347%


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.354000,0.308009,0.921158,0.924989,0.946790,0.904170
2,0.320100,0.302928,0.927097,0.932285,0.931129,0.933444
3,0.319500,0.291822,0.934670,0.939676,0.933025,0.946424
4,0.258800,0.311290,0.933333,0.938518,0.930744,0.946424
5,0.276500,0.317225,0.938085,0.941160,0.962204,0.921016
6,0.277200,0.292271,0.941945,0.945536,0.953907,0.937310
7,0.267400,0.294838,0.940757,0.944744,0.947500,0.942005
8,0.242800,0.335551,0.933482,0.939394,0.920711,0.958851
9,0.242900,0.303215,0.944024,0.947486,0.955874,0.939243
10,0.232000,0.307135,0.942390,0.946453,0.945931,0.946976



[Early Stopping] No improvement over best F1 for 3 consecutive epochs. Stopping training at epoch 14.
[Early Stopping Info] Best F1: 0.9486 at epoch 11


[GPU Info] Used 1 GPU(s), Current device: 0 (NVIDIA RTX 5000 Ada Generation)



Results for rank = 16:
  Test Accuracy: 0.9436
  Test F1: 0.9495
  Validation Accuracy: 0.9451
  Validation F1: 0.9486
  Total Training Time: 2025.79s
  Average Epoch Time: 139.21s
  Best F1 Epoch: 11
  Best Val F1: 0.9486
  Early Stopped: True
  Stop Info: epochs_without_improvement=3
  Peak GPU Memory: 604.68 MB
  LoRA Sparsity (<1e-3): 22.17%

[Rank 16] Saved model to /hpc/group/xielab/hl385/LoRA/sparse_lora_rank16/final_model
[Rank 16] Saved log history to log_history.json


GENERATING SUMMARY TABLE

BENCHMARK SUMMARY - Sparse LoRA
 Rank Test Acc Test F1 Val Acc Val F1 Best Val F1  Best F1 Epoch Trainable Params Trainable % Total Time (s) Avg Epoch (s) Early Stopped Peak GPU (MB) Sparsity (<1e-3)
    2   0.9424  0.9487  0.9422 0.9461      0.9461              8          665,858       0.98%        1627.23        142.58           Yes        590.62            2.86%
    4   0.9474  0.9534  0.9476 0.9511      0.9511             20          739,586       1.09%        3373.55        140.9

### IMDB Dataset

Have commented out in the corresponding section \
Just to replace the loaded dataset accordingly:
- DATASET = "imdb"
- TEXT_COL = "text"
- LABEL_COL = "label"